In [1]:
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers
from vocab import Vocab
from helper_functions import *
from dataset import *

f:\anaconda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
f:\anaconda\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
f:\anaconda\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
vocab_size=23
batch_size=64
embedding_dim=256
rnn_units=256

In [3]:
df_General_data = pd.read_csv("data\general_peptide_data.csv")
vocabulary, _ = Vocab.create_vocab(np.array(df_General_data.Sequence.values))

In [4]:
vocab = vocabulary.vocab
print(vocab)

{'!': 1, 'G': 8, 'D': 5, 'V': 20, 'K': 11, 'F': 7, 'S': 18, 'L': 12, '%': 2, 'H': 9, 'N': 14, 'Y': 22, 'I': 10, 'E': 6, 'Q': 16, 'C': 4, 'R': 17, 'W': 21, 'P': 15, 'A': 3, 'T': 19, 'M': 13, '+': 0}


In [5]:
x = tf.keras.Input(shape=(None,), batch_size = 1)
x2 = tf.keras.layers.Embedding(
    input_dim=vocab_size, output_dim=embedding_dim, mask_zero=True
)(x)
h1 = layers.GRU(rnn_units, return_sequences=True, stateful=True)(x2)
h2 = layers.GRU(rnn_units, return_sequences=True, stateful=True)(h1)

yhat = tf.keras.layers.Dense(vocab_size)(h2)
inference_model = tf.keras.Model(inputs=x, outputs=yhat)
inference_model.load_weights("GRU_TL\checkpoint")

In [6]:
inference_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(1, None)]               0         
                                                                 
 embedding (Embedding)       (1, None, 256)            5888      
                                                                 
 gru (GRU)                   (1, None, 256)            394752    
                                                                 
 gru_1 (GRU)                 (1, None, 256)            394752    
                                                                 
 dense (Dense)               (1, None, 23)             5911      
                                                                 
Total params: 801,303
Trainable params: 801,303
Non-trainable params: 0
_________________________________________________________________


In [7]:
pep_gen_base = Generate(inference_model,vocab)

In [8]:
generated_seqs = pep_gen_base.generate_multi_seqs(50000,8, 1)

100%|██████████| 50000/50000 [5:31:23<00:00,  2.51it/s]  


In [14]:
gen_df = pd.DataFrame(generated_seqs, columns = ["generated_seqs"])

In [16]:
gen_df.to_pickle("data/Generated_unfilter.pkl")

In [17]:
gen_df.to_csv("data/Generated_unfilter.csv")